In [ ]:
#| default_exp datastructure.ts_format

In [ ]:
#| hide

from IPython.core.debugger import set_trace

%load_ext autoreload
%autoreload 2

# Time Series (ts) format 
> Transform a pandas Dataframe to ts format.

In [ ]:
#| export
import pandas as pd
from pathlib import Path
from typing import Dict
from soccer_prediction_data.datastructure.data_extractor import STATS

In [ ]:
#| export

FILE_METADATA = {
    "source": "Instat",
    "creator": "Real-Analytics",
    "data_description": """More than 5 years of football data from leagues across the world that includes stats from both competing teams in each played game in each dimension.\n#The data describes a series of numbers achieved by each squad, which indicate their athletic performance.\n#This yields a database of almost 52k time series.""",
    "task": "The task is to  predict the outcomes of a set of soccer matches from leagues worldwide.",
    "problem_name": "soccer-preds",
    "time_stamps": "false",
    "missing": "false",
    "univariate": "false",
    "dimensions": 2,
    "equal_length": "true",
    "class_label": "true 0 1 2",
}

In [ ]:
# | export

def create_ts_file(
    df: pd.DataFrame,  # Pandas Dataframe input.
    file_path: str = ".",  # Where should we save our file ??.
    file_name: str = "games",  # File name.
    file_metadata: Dict = FILE_METADATA,  # File metadata.
) -> None:
    "Create a ts file from a Pandas dataframe."

    # Check path.
    if not Path(file_path).exists():
        raise Exception("File path does not exist.")

    # Create an empty ts file.
    with open(f"{file_path}/{file_name}.ts", "w") as f:
        # Add data length for each dimension.
        file_metadata["series_length"] = len(STATS)
        # Init header file information.
        header = "\n".join(
            (
                f'#Source: {file_metadata["source"]}',
                f'#Creator: {file_metadata["creator"]}',
                "#",
                "#Data Set Information:",
                "#",
                f'#{file_metadata["data_description"]}',
                "#",
                f'#{file_metadata["task"]}',
                f'@problemName {file_metadata["problem_name"]}',
                f'@timeStamps {file_metadata["time_stamps"]}',
                f'@missing {file_metadata["missing"]}',
                f'@univariate {file_metadata["univariate"]}',
                f'@dimensions {file_metadata["dimensions"]}',
                f'@equalLength {file_metadata["equal_length"]}',
                f'@seriesLength {file_metadata["series_length"]}',
                f'@classLabel {file_metadata["class_label"]}',
                f"@data",
            )
        )
        # Add header file information.
        f.write(header)
        # Loop over data to extract info.
        for _, row in df.iterrows():
            # Extract the values for each dimension and store them in separate lists.
            # First dimension for home team features.
            dim1_vals = list(row.filter(like="homeTeam")[2:])
            # Second dimension for home team features.
            dim2_vals = list(row.filter(like="awayTeam")[2:])
            # Map results {homewin -> 0 , draw -> 1, awaywin -> 2}.
            target_val = 0 if row.HS > row.AS else 2 if row.HS < row.AS else 1

            # Combine the values for each dimension into a single string.
            data_str = f"{','.join(str(val) for val in dim1_vals)}:{','.join(str(val) for val in dim2_vals)}:{target_val}"

            # Write the string to the ts file.
            f.write("\n"+ data_str + "\n")

In [ ]:
games = pd.read_csv("games.csv").fillna(0)
create_ts_file(
    df=games,
    file_path=".",
    file_name="games",
    file_metadata=FILE_METADATA,
)

In [ ]:
#| hide

import nbdev

nbdev.nbdev_export()